# Notebook 10: Neural vs QML Comparison

**Purpose**: Compare neural network models with quantum ML models.

**Inputs**:
- `neural_metrics.csv`
- `qml_metrics.csv`

**Outputs**:
- Comparison visualizations → `figures/`

---

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Paths
BASE_DIR = Path('.').resolve().parent
RESULTS_DIR = BASE_DIR / 'results'
FIGURES_DIR = BASE_DIR / 'figures'

# Style
plt.style.use('seaborn-v0_8-whitegrid')
colors = {'neural': '#e74c3c', 'qml': '#9b59b6'}

In [ ]:
# Load metrics
neural_df = pd.read_csv(RESULTS_DIR / 'neural_metrics.csv')
qml_df = pd.read_csv(RESULTS_DIR / 'qml_metrics.csv')

neural_df['category'] = 'Neural'
qml_df['category'] = 'QML'

combined_df = pd.concat([neural_df, qml_df], ignore_index=True)

print(f"Neural models: {len(neural_df)}")
print(f"QML models: {len(qml_df)}")

In [ ]:
# Display results
print("\nNeural Models:")
print(neural_df[['model', 'f1_score', 'roc_auc', 'train_time']].to_string(index=False))
print("\nQML Models:")
print(qml_df[['model', 'f1_score', 'roc_auc', 'train_time', 'n_qubits']].to_string(index=False))

## Performance Comparison

In [ ]:
# Metric comparison
metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, metric in enumerate(metrics):
    ax = axes[i]
    data = combined_df.sort_values(metric, ascending=True)
    colors_list = [colors['neural'] if c == 'Neural' else colors['qml'] 
                   for c in data['category']]
    
    ax.barh(data['model'], data[metric], color=colors_list)
    ax.set_xlabel(metric.replace('_', ' ').title())
    ax.set_title(f'{metric.replace("_", " ").title()} Comparison')
    ax.set_xlim([0, 1])

from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=colors['neural'], label='Neural'),
                   Patch(facecolor=colors['qml'], label='QML')]
axes[-1].legend(handles=legend_elements, loc='center')
axes[-1].axis('off')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'neural_vs_qml_metrics.png', dpi=150)
plt.show()

## Scalability Analysis

In [ ]:
# Performance vs Training Time scatter
fig, ax = plt.subplots(figsize=(10, 8))

for category, color in colors.items():
    if category == 'neural':
        data = neural_df
    else:
        data = qml_df
    
    ax.scatter(data['train_time'], data['f1_score'], 
               c=color, label=category.upper(), s=100, alpha=0.7)
    
    for _, row in data.iterrows():
        ax.annotate(row['model'], (row['train_time'], row['f1_score']),
                   fontsize=8, ha='left', va='bottom')

ax.set_xlabel('Training Time (seconds)')
ax.set_ylabel('F1 Score')
ax.set_title('Performance vs Training Time: Neural vs QML')
ax.legend()
ax.set_xscale('log')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'neural_vs_qml_scalability.png', dpi=150)
plt.show()

In [ ]:
# QML-specific: Performance vs Qubits
if 'n_qubits' in qml_df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    ax1 = axes[0]
    ax1.scatter(qml_df['n_qubits'], qml_df['f1_score'], s=100, c=colors['qml'])
    for _, row in qml_df.iterrows():
        ax1.annotate(row['model'], (row['n_qubits'], row['f1_score']), fontsize=8)
    ax1.set_xlabel('Number of Qubits')
    ax1.set_ylabel('F1 Score')
    ax1.set_title('QML: Performance vs Qubit Count')
    
    ax2 = axes[1]
    ax2.scatter(qml_df['n_qubits'], qml_df['train_time'], s=100, c=colors['qml'])
    for _, row in qml_df.iterrows():
        ax2.annotate(row['model'], (row['n_qubits'], row['train_time']), fontsize=8)
    ax2.set_xlabel('Number of Qubits')
    ax2.set_ylabel('Training Time (s)')
    ax2.set_title('QML: Training Time vs Qubit Count')
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'qml_qubit_analysis.png', dpi=150)
    plt.show()

In [ ]:
# Summary
print("\n" + "="*60)
print("NEURAL VS QML SUMMARY")
print("="*60)

for metric in ['f1_score', 'roc_auc', 'train_time']:
    neural_mean = neural_df[metric].mean()
    qml_mean = qml_df[metric].mean()
    
    print(f"\n{metric.replace('_', ' ').title()}:")
    print(f"  Neural: {neural_mean:.4f}")
    print(f"  QML: {qml_mean:.4f}")
    
    if metric != 'train_time':
        winner = 'Neural' if neural_mean > qml_mean else 'QML'
    else:
        winner = 'Neural' if neural_mean < qml_mean else 'QML'
    print(f"  Winner: {winner}")

print("\n✅ Notebook 10 Complete!")